In [2]:
import folium
import pandas as pd
import numpy as np


In [4]:
peregon = pd.read_excel("D:\MyFolder\\2.Work\DataWagon\PEREGON_HACKATON.xlsx")
station_coord = pd.read_excel("D:\MyFolder\\2.Work\DataWagon\STATION_COORDS_HACKATON.xlsx")

In [5]:
peregon

,START_CODE,END_CODE,LEN
0,2,805,32
1,6,2,27
2,8,6,27
3,12,8,29
4,15,12,19
...,...,...,...
8969,24329,7173,9
8970,25137,5956,10
8971,25137,5985,1
8972,25612,1688,0


In [6]:
station_coord_cleaned = station_coord.dropna()


In [7]:
station_coord

,ST_ID,LATITUDE,LONGITUDE
0,2,48.4272,42.2162
1,3,48.4910,42.3610
2,4,48.3981,42.0699
3,5,48.3857,41.9546
4,6,48.5423,42.5028
...,...,...,...
15168,25876,56.5624,57.7346
15169,25877,56.6841,57.9468
15170,25878,56.8725,59.2641
15171,25879,56.4072,52.9355


In [8]:
station_coord_cleaned

,ST_ID,LATITUDE,LONGITUDE
0,2,48.4272,42.2162
1,3,48.4910,42.3610
2,4,48.3981,42.0699
3,5,48.3857,41.9546
4,6,48.5423,42.5028
...,...,...,...
15168,25876,56.5624,57.7346
15169,25877,56.6841,57.9468
15170,25878,56.8725,59.2641
15171,25879,56.4072,52.9355


In [9]:
# Создаем карту
m = folium.Map(location=[station_coord_cleaned['LATITUDE'].mean(), station_coord_cleaned['LONGITUDE'].mean()], zoom_start=5)

# Добавляем статичные маркеры (CircleMarker) для каждого объекта
for index, row in station_coord_cleaned.iterrows():
    folium.CircleMarker(location=[row['LATITUDE'], row['LONGITUDE']],
                        radius=5,  # радиус круга (можно настроить)
                        color='blue',  # цвет круга
                        fill=True,
                        fill_color='blue',  # цвет заливки
                        fill_opacity=0.7,  # прозрачность заливки
                        popup=row['ST_ID']).add_to(m)

# Сохраняем карту в HTML-файл
m.save('Station_coord_map.html')


In [10]:
# Создаем карту
m = folium.Map(location=[station_coord_cleaned['LATITUDE'].mean(), station_coord_cleaned['LONGITUDE'].mean()], zoom_start=5)

# Добавляем линии между соответствующими вершинами
for index, row in peregon.iterrows():
    start_station = station_coord_cleaned.loc[station_coord_cleaned['ST_ID'] == row['START_CODE']]
    end_station = station_coord_cleaned.loc[station_coord_cleaned['ST_ID'] == row['END_CODE']]
    
    if not start_station.empty and not end_station.empty:
        start_coords = [start_station['LATITUDE'].values[0], start_station['LONGITUDE'].values[0]]
        end_coords = [end_station['LATITUDE'].values[0], end_station['LONGITUDE'].values[0]]
        
        folium.PolyLine(locations=[start_coords, end_coords], color='blue', weight=2, opacity=0.7).add_to(m)

# Сохраняем карту в HTML-файл
m.save('railway_lines_map.html')

In [11]:
disl = pd.read_excel("D:\MyFolder\\2.Work\DataWagon\disl_hackaton.xlsx")

In [12]:
disl

,WAGNUM,OPERDATE,ST_ID_DISL,ST_ID_DEST,TRAIN_INDEX
0,5266,2023-08-30 01:02:00,7475,61,7475-335-62
1,5266,2023-08-30 05:26:00,63,61,7475-335-62
2,5266,2023-08-30 05:05:00,7475,61,7475-335-62
3,5266,2023-08-28 23:45:00,7475,61,6999-471-7475
4,5266,2023-08-28 22:29:00,7469,61,6999-471-7475
...,...,...,...,...,...
999994,5800,2023-07-15 03:47:00,2284,2331,16069-199-2331
999995,7034,2023-07-15 03:47:00,2284,2331,16069-199-2331
999996,7034,2023-07-15 02:08:00,2276,2331,16069-199-2331
999997,5800,2023-07-15 02:08:00,2276,2331,16069-199-2331


In [13]:
# Найти строки с NaN значениями
nan_rows = disl[disl.isnull().any(axis=1)]

# Вывести найденные строки
print(nan_rows)


        WAGNUM            OPERDATE  ST_ID_DISL  ST_ID_DEST TRAIN_INDEX
282814    6167 2023-06-12 23:50:00        9465        1227         NaN
282820    6167 2023-06-09 14:03:00       16947        1227         NaN
283042    6167 2023-06-11 18:52:00        9435        1227         NaN
283143    6167 2023-06-10 19:52:00        9448        1227         NaN
283650    6273 2023-06-12 23:50:00        9465        1227         NaN
283653    6273 2023-06-09 14:03:00       16947        1227         NaN
283980    6273 2023-06-11 18:52:00        9435        1227         NaN
284097    6273 2023-06-10 19:52:00        9448        1227         NaN
287940    7173 2023-06-12 23:50:00        9465        1227         NaN
287945    7173 2023-06-09 14:03:00       16947        1227         NaN
288283    7173 2023-06-11 18:52:00        9435        1227         NaN
288440    7173 2023-06-10 19:52:00        9448        1227         NaN
332566    9765 2023-07-22 14:00:00       20695        1561         NaN
332582

In [14]:
# Сортировка данных по столбцу OPERDATE
sorted_df = disl.sort_values(by='OPERDATE')

# Группировка данных по столбцу OPERDATE
grouped_df = sorted_df.groupby('OPERDATE')


In [15]:
first_data_input = sorted_df[0:100].reset_index()

In [16]:
first_data_input

,index,WAGNUM,OPERDATE,ST_ID_DISL,ST_ID_DEST,TRAIN_INDEX
0,340706,6516,2023-06-01 00:00:00,1268,1655,1268-677-1663
1,339038,5100,2023-06-01 00:00:00,1268,1655,1268-677-1663
2,340705,6516,2023-06-01 00:00:01,1268,1655,1268-677-1663
3,339037,5100,2023-06-01 00:00:01,1268,1655,1268-677-1663
4,88525,6394,2023-06-01 00:01:00,4576,633,20715-079-20713
...,...,...,...,...,...,...
95,984338,3041,2023-06-01 00:24:00,13913,2331,16069-096-2331
96,923814,8966,2023-06-01 00:24:00,13913,2331,16069-096-2331
97,88527,6394,2023-06-01 00:24:00,4571,633,20715-079-20713
98,955631,8482,2023-06-01 00:26:00,1839,2331,16069-088-2331


In [20]:


# Преобразуем столбцы с датами в формат datetime
first_data_input['OPERDATE'] = pd.to_datetime(first_data_input['OPERDATE'])

# 1. Взять самую новую информацию по каждому вагону
latest_data = first_data_input.sort_values(by='OPERDATE', ascending=False).groupby('WAGNUM').head(1)

# 2. Отметить на карте текущее расположение вагона
m = folium.Map(location=[station_coord['LATITUDE'].mean(), station_coord['LONGITUDE'].mean()], zoom_start=5)

for index, row in latest_data.iterrows():
    # Добавляем маркер текущего расположения вагона
    current_location = station_coord.loc[station_coord['ST_ID'] == row['ST_ID_DISL']]
    if not current_location.empty:
        folium.Marker(location=[current_location['LATITUDE'].values[0], current_location['LONGITUDE'].values[0]],
                    popup=f"Train: {row['TRAIN_INDEX']}, Destination: {row['ST_ID_DEST']}").add_to(m)

# 3. При нажатии на вагон отметить точку назначения текущего вагона
for index, row in latest_data.iterrows():
    # Добавляем всплывающий маркер для точки назначения
    destination_location = station_coord.loc[station_coord['ST_ID'] == row['ST_ID_DEST']]
    if not destination_location.empty:
        folium.Marker(location=[destination_location['LATITUDE'].values[0], destination_location['LONGITUDE'].values[0]],
                    popup=f"Destination: {row['ST_ID_DEST']}").add_to(m)

# Сохраняем карту в HTML-файл
m.save('train_map.html')


In [21]:
# Предположим, что first_data_input - это ваш исходный датафрейм
# Сортируем по дате операции в обратном порядке и выбираем первую запись для каждого вагона
latest_data = first_data_input.sort_values(by='OPERDATE', ascending=False).groupby('WAGNUM').first().reset_index()


In [24]:

# Создаем карту
m = folium.Map(location=[station_coord['LATITUDE'].mean(), station_coord['LONGITUDE'].mean()], zoom_start=5)

# Добавляем линии между соответствующими вершинами
for index, row in peregon.iterrows():
    start_station = station_coord.loc[station_coord['ST_ID'] == row['START_CODE']]
    end_station = station_coord.loc[station_coord['ST_ID'] == row['END_CODE']]
    
    if not start_station.empty and not end_station.empty:
        start_coords = [start_station['LATITUDE'].values[0], start_station['LONGITUDE'].values[0]]
        end_coords = [end_station['LATITUDE'].values[0], end_station['LONGITUDE'].values[0]]
        
        folium.PolyLine(locations=[start_coords, end_coords], color='blue', weight=2, opacity=0.7).add_to(m)

# Добавляем маркеры для текущего расположения вагона
for index, row in latest_data.iterrows():
    station = station_coord.loc[station_coord['ST_ID'] == row['ST_ID_DISL']]
    if not station.empty:
        coords = [station['LATITUDE'].values[0], station['LONGITUDE'].values[0]]
        folium.Marker(location=coords, popup=f"Wagon: {row['WAGNUM']}").add_to(m)

# Сохраняем карту в HTML-файл
m.save('wagon_map.html')


In [27]:
# Добавляем маркеры для точек назначения вагонов
for index, row in latest_data.iterrows():
    destination_station = station_coord.loc[station_coord['ST_ID'] == row['ST_ID_DEST']]
    if not destination_station.empty:
        destination_coords = [destination_station['LATITUDE'].values[0], destination_station['LONGITUDE'].values[0]]
        
        # Создаем маркер с точкой назначения
        destination_marker = folium.Marker(location=destination_coords, popup=f"Destination for Wagon {row['WAGNUM']}")
        
        # Создаем всплывающее окно с информацией по вагону при нажатии
        destination_marker.add_child(folium.Popup(f"Wagon: {row['WAGNUM']}<br>Train Index: {row['TRAIN_INDEX']}"))
        
        # Добавляем маркер точки назначения на карту
        destination_marker.add_to(m)

# Сохраняем карту в HTML-файл
m.save('wagon_map_with_destination.html')


In [29]:
# Создаем карту
m = folium.Map(location=[station_coord['LATITUDE'].mean(), station_coord['LONGITUDE'].mean()], zoom_start=5)

# Добавляем линии между соответствующими вершинами
for index, row in peregon.iterrows():
    start_station = station_coord.loc[station_coord['ST_ID'] == row['START_CODE']]
    end_station = station_coord.loc[station_coord['ST_ID'] == row['END_CODE']]
    
    if not start_station.empty and not end_station.empty:
        start_coords = [start_station['LATITUDE'].values[0], start_station['LONGITUDE'].values[0]]
        end_coords = [end_station['LATITUDE'].values[0], end_station['LONGITUDE'].values[0]]
        
        folium.PolyLine(locations=[start_coords, end_coords], color='blue', weight=2, opacity=0.7).add_to(m)

# Добавляем маркеры для текущего расположения вагона
for index, row in latest_data.iterrows():
    station = station_coord.loc[station_coord['ST_ID'] == row['ST_ID_DISL']]
    if not station.empty:
        coords = [station['LATITUDE'].values[0], station['LONGITUDE'].values[0]]
        
        # Создаем маркер текущего расположения вагона
        marker = folium.Marker(location=coords, popup=f"Wagon: {row['WAGNUM']}")
        
        # Добавляем обработчик события для отображения маркеров точек назначения при клике на маркер текущего расположения
        marker.add_child(folium.Popup(f"""<button onclick="showDestinationMarkers('{row['WAGNUM']}')">
                                            Show Destination for Wagon {row['WAGNUM']}
                                         </button>"""))
        marker.add_to(m)

# Добавляем маркеры точек назначения (скрытыми)
for index, row in latest_data.iterrows():
    destination_station = station_coord.loc[station_coord['ST_ID'] == row['ST_ID_DEST']]
    if not destination_station.empty:
        destination_coords = [destination_station['LATITUDE'].values[0], destination_station['LONGITUDE'].values[0]]
        destination_marker = folium.Marker(location=destination_coords,
                                           popup=f"Destination for Wagon {row['WAGNUM']}",
                                           icon=folium.Icon(color='red'),
                                           display='none')  # Скрываем маркер
        destination_marker.add_to(m)

# JavaScript-функция для отображения маркеров точек назначения
js_code = """
    function showDestinationMarkers(wagonNumber) {
        // Получаем все маркеры точек назначения для данного вагона
        var destinationMarkers = document.querySelectorAll('[id^="destination_marker_' + wagonNumber + '"]');
        
        // Перебираем и показываем каждый маркер
        destinationMarkers.forEach(function(marker) {
            marker.style.display = 'block';
        });
    }
"""
folium.Map().add_child(folium.Element(js_code))

# Сохраняем карту в HTML-файл
m.save('wagon_map_with_destination.html')
